In [1]:
import pandas as pd
import numpy as np

In [2]:
with open('../data/latest_timestamp.txt', 'r') as f:
    latest_timestamp = f.read().strip()

file_path = f"../data/processed/cleaned_movies_{latest_timestamp}.parquet"
df = pd.read_parquet(file_path, engine='pyarrow')
df.head()

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,...,runtime,overview,spoken_languages,poster_path,cast,cast_size,director,crew_size,profit_musd,roi
0,299534,Avengers: Endgame,Avenge the fallen.,2019-04-24,Adventure | Science Fiction | Action,The Avengers Collection,en,356.0,2799.439100,Marvel Studios,...,181,After the devastating events of Avengers: Infi...,en | ja | xh,/ulzhLuWrPK07P1YkdWQLZnQh1JL.jpg,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",105,"Anthony Russo, Joe Russo",593,2443.44,686.36
1,19995,Avatar,Enter the world of Pandora.,2009-12-15,Action | Adventure | Fantasy | Science Fiction,Avatar Collection,en,237.0,2923.706026,Dune Entertainment | Lightstorm Entertainment ...,...,162,"In the 22nd century, a paraplegic Marine is di...",en | es,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,"Sam Worthington, Zoe Saldaña, Sigourney Weaver...",65,James Cameron,986,2686.71,1133.63
2,140607,Star Wars: The Force Awakens,Every generation has a story.,2015-12-15,Adventure | Action | Science Fiction,Star Wars Collection,en,245.0,2068.223624,Lucasfilm Ltd. | Bad Robot,...,136,Thirty years after defeating the Galactic Empi...,en,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,"Harrison Ford, Mark Hamill, Carrie Fisher, Ada...",182,J.J. Abrams,257,1823.22,744.17
3,299536,Avengers: Infinity War,Destiny arrives all the same.,2018-04-25,Adventure | Action | Science Fiction,The Avengers Collection,en,300.0,2052.415039,Marvel Studios,...,149,As the Avengers and their allies have continue...,en | xh,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,"Robert Downey Jr., Chris Evans, Chris Hemswort...",69,"Anthony Russo, Joe Russo",724,1752.42,584.14
4,597,Titanic,Nothing on Earth could come between them.,1997-11-18,Drama | Romance,NaN,en,200.0,2264.162353,Paramount Pictures | 20th Century Fox | Lights...,...,194,101-year-old Rose DeWitt Bukater tells the sto...,en | fr | de | sv | it | ru,/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,"Leonardo DiCaprio, Kate Winslet, Billy Zane, K...",116,James Cameron,258,2064.16,1032.08


In [3]:
def rank_movies(df, metric, ascending=False, min_votes=10, min_budget=10):
    filtered_df = df.copy()
    if 'vote' in metric.lower():
        filtered_df = filtered_df[filtered_df['vote_count'] >= min_votes]
    if 'roi' in metric.lower() or 'profit' in metric.lower():
        filtered_df = filtered_df[filtered_df['budget_musd'] >= min_budget]
    filtered_df = filtered_df.sort_values(by=metric, ascending=ascending)
    return filtered_df

def perform_analysis(df):
    df['profit_musd'] = (df['revenue_musd'] - df['budget_musd']).round(2)
    df['roi'] = (df['revenue_musd'] - df['budget_musd']).round(2)

    # Identify best and worst movies 
    kpis = {
        'highest_revenue': rank_movies(df, 'revenue_musd').head(5),
        'highest_budget': rank_movies(df, 'budget_musd').head(5),
        'highest_profit': rank_movies(df, 'profit_musd').head(5),
        'lowest_profit': rank_movies(df, 'profit_musd', ascending=True).head(5),
        'highest_roi': rank_movies(df, 'roi').head(5),
        'lowest_roi': rank_movies(df, 'roi', ascending=True).head(5),
        'most_voted': rank_movies(df, 'vote_count', min_votes=0).head(5),
        'highest_rated': rank_movies(df, 'vote_average').head(5),
        'lowest_rated': rank_movies(df, 'vote_average', ascending=True).head(5),
        'most_popular': rank_movies(df, 'popularity', min_votes=0).head(5),
    }

    # Best rated Sci-fi action movies with Bruce Willis
    sci_fi_action_bruce_willis = df[
        df['genres'].str.contains('Science Fiction', na=False) &
        df['genres'].str.contains('Action', na=False) &
        df['cast'].str.contains('Bruce Willis', na=False)
    ].sort_values(by='vote_average', ascending=False)

    # Movies with Uma Thurman directed by Quentin Tarantino
    uma_thurman_tarentino_directed = df[
        df['cast'].str.contains('Uma Thurman', na=False) &
        (df['director'] == 'Quentin Tarantino')
    ].sort_values(by='runtime')

    # Franchise vs Standalone Movie Performance
    franchise_df = df[df['belongs_to_collection'].notna()]
    standalone_df = df[df['belongs_to_collection'].isna()]

    franchise_stats = {
        'mean_revenue': franchise_df['revenue_musd'].mean(),
        'median_roi': franchise_df['roi'].median(),
        'mean_budget': franchise_df['budget_musd'].mean(),
        'mean_popularity': franchise_df['popularity'].mean(),
        'mean_rating': franchise_df['vote_average'].mean()
    }

    standalone_stats = {
        'mean_revenue': standalone_df['revenue_musd'].mean(),
        'median_roi': standalone_df['roi'].median(),
        'mean_budget': standalone_df['budget_musd'].mean(),
        'mean_popularity': standalone_df['popularity'].mean(),
        'mean_rating': standalone_df['vote_average'].mean(),
    }

    # Most Successful Franchises
    franchise_performance = franchise_df.groupby('belongs_to_collection', observed=False).agg({
        'title': 'count',
        'budget_musd': ['sum', 'mean'],
        'revenue_musd': ['sum', 'mean'],
        'vote_average': 'mean'
    }).sort_values(by=('revenue_musd', 'sum'), ascending=False)

    franchise_performance.columns = [
        'num_movies', 'total_budget_musd', 'mean_budget_musd',
        'total_revenue_musd', 'mean_revenue_musd', 'mean_rating'
    ]

    # Most Successful Directors
    director_performance = df.groupby('director').agg({
        'title': 'count',
        'revenue_musd': ['sum'],
        'vote_average': 'mean'
    }).sort_values(by=('revenue_musd', 'sum'), ascending=False)

    director_performance.columns = [
        'num_movies', 'total_revenue_musd', 'mean_rating'
    ]
    
    return {
        'kpis': kpis,
        'sci_fi_action_bruce_willis': sci_fi_action_bruce_willis,
        'uma_thurman_tarentino_directed': uma_thurman_tarentino_directed,
        'franchise_stats': franchise_stats,
        'standalone_stats': standalone_stats,
        'franchise_performance': franchise_performance,
        'director_performance': director_performance
    }

if __name__ == "__main__":
    analysis_results = perform_analysis(df)

    # Print summaries
    print("Top 5 Highest Revenue Movies:")
    print(analysis_results['kpis']['highest_revenue'][['title', 'revenue_musd']])

    print("Highest budget movies:")
    print(analysis_results['kpis']['highest_budget'][['title', 'budget_musd']])

    print("Highest profit movies:")
    print(analysis_results['kpis']['highest_profit'][['title', 'profit_musd']])

    print("Lowest profit movies:")
    print(analysis_results['kpis']['lowest_profit'][['title', 'profit_musd']])

    print("Highest ROI movies:")
    print(analysis_results['kpis']['highest_roi'][['title', 'roi']])

    print("Lowest ROI movies:")
    print(analysis_results['kpis']['lowest_roi'][['title', 'roi']])   

    print("Most voted movies:")
    print(analysis_results['kpis']['most_voted'][['title', 'vote_count']])

    print("Highest rated movies:")
    print(analysis_results['kpis']['highest_rated'][['title', 'vote_average']])

    print("Lowest rated movies:")
    print(analysis_results['kpis']['lowest_rated'][['title', 'vote_average']])

    print("Most popular movies:")
    print(analysis_results['kpis']['most_popular'][['title', 'popularity']])                                            

    print("\nBruce Willis Sci-Fi Action Movies:")
    print(analysis_results['sci_fi_action_bruce_willis'][['title', 'vote_average']])

    print("\nUma Thurman & Tarantino Movies:")
    print(analysis_results['uma_thurman_tarentino_directed'][['title', 'runtime']])

    print("\nFranchise Stats:")
    print(analysis_results['franchise_stats'])

    print("\nStandalone Stats:")
    print(analysis_results['standalone_stats'])

    print("\nTop 5 Franchises by Total Revenue:")
    print(analysis_results['franchise_performance'].head(5))

    print("\nTop 5 Directors by Total Revenue:")
    print(analysis_results['director_performance'].head(5))

    df.to_parquet(f"../data/processed/cleaned_movies_{latest_timestamp}.parquet", engine='pyarrow', index=False)  # Save the updated DataFrame
    print("DataFrame saved successfully.")


Top 5 Highest Revenue Movies:
                          title  revenue_musd
1                        Avatar   2923.706026
0             Avengers: Endgame   2799.439100
4                       Titanic   2264.162353
2  Star Wars: The Force Awakens   2068.223624
3        Avengers: Infinity War   2052.415039
Highest budget movies:
                          title  budget_musd
9       Avengers: Age of Ultron        365.0
0             Avengers: Endgame        356.0
3        Avengers: Infinity War        300.0
6                 The Lion King        260.0
2  Star Wars: The Force Awakens        245.0
Highest profit movies:
                          title  profit_musd
1                        Avatar      2686.71
0             Avengers: Endgame      2443.44
4                       Titanic      2064.16
2  Star Wars: The Force Awakens      1823.22
3        Avengers: Infinity War      1752.42
Lowest profit movies:
                       title  profit_musd
9    Avengers: Age of Ultron      1040.40
17